In [14]:
import pandas as pd
import cv2
import multiprocessing
from functools import partial
import itertools
import os
import numpy as np

In [3]:
import sys
sys.path.append('../')

In [4]:
from model.scripts.packages.yolo_predict import YOLO_Pred

In [6]:
yolo = YOLO_Pred('/home/cremerf/FinalProject/data/weights/first_training/weights/bestnoft.onnx', '/home/cremerf/FinalProject/data/config_blmodel.yaml')

In [12]:
def get_neightbours(df_predictions:pd.DataFrame, neightbour:str, index_it:int) -> dict:
    """
    Identifies the neightbours of each predicted bounding box from a given image.
    
    Takes as parameter a dataframe which stores all the coords of the predicted bounding boxes.
    Defines 4 filters to limit the spectrum to look for the neightbours. 
    
    Each filter is calculated from X_center/Y_center.

    You have to specify which side(right or left) you want to evaluate. 

    Returns a dictionary with key:pair values, where keys are the evaluted bounding box and the pair values are the neightbours of that key value.

    This function is structured in a way to be suited for parallelizing. 
    
    The goal is to optimize times frames and resources to get the left/right neightbours as fast as possible

    Args:
        df_predictions (pd.DataFrame): Dataframe with predicted bb obtained with Yolov5.
        neightbour (str): Right or Left.
        index_it (int): _description_

    Returns:
        dict_of_neightbours: Bounding_box:BB_Neightbours
    """

    dict_of_neightbours = {} 

    X_center_0 = df_predictions.loc[index_it][0]
    Y_center_0 = df_predictions.loc[index_it][1]
    Width_0 = df_predictions.loc[index_it][2]
    height_0 = df_predictions.loc[index_it][3]
    threshold_x_0 = 1 * Width_0
    threshold_y_0 = 1 * height_0

    INPUT_WH_YOLO = 640
    filter_1 = (df_predictions.loc[:, 'X_center'] < (X_center_0 + INPUT_WH_YOLO/2))
    filter_2 = (df_predictions.loc[:, 'X_center'] > (X_center_0 - INPUT_WH_YOLO/2))
    filter_3 = (df_predictions.loc[:, 'Y_center'] > (Y_center_0 - INPUT_WH_YOLO/2))
    filter_4 = (df_predictions.loc[:, 'Y_center'] < (Y_center_0 + INPUT_WH_YOLO/2))
    filter_final = (filter_1 & filter_2) & (filter_3 & filter_4)

    df_predictions_final_2 = df_predictions[filter_final]

    for index_bb in df_predictions_final_2.index:

        list_of_neightbours_l = []
        list_of_neightbours_r = []
        list_of_alones_l = []
        list_of_alones_r = []

        X_center_neightbour = df_predictions.loc[index_bb][0]
        Y_center_neightbour = df_predictions.loc[index_bb][1]
        a = 2
        k = 2

        ### Neightbor Left
        if neightbour == 'left':
            
            x_min_l = X_center_0 - Width_0 - (a * threshold_x_0)
            x_max_l = X_center_0 - Width_0 + (k * threshold_x_0)

            y_min_l = Y_center_0 - (k * threshold_y_0)
            y_max_l = Y_center_0 + (k * threshold_y_0)

            if (x_min_l < X_center_neightbour < x_max_l and y_min_l < Y_center_neightbour < y_max_l )  :
                list_of_neightbours_l.append([index_bb])
                try:
                    dict_of_neightbours[str(index_it)+'_l'].append(index_bb)
                except KeyError:
                    dict_of_neightbours[str(index_it)+'_l']= []
                    dict_of_neightbours[str(index_it)+'_l'].append(index_bb)
            else:
                list_of_alones_l.append([index_bb])

        ### Neightbor Right
        elif neightbour == 'right':

            x_min_r = X_center_0 + Width_0 - (k*threshold_x_0)
            x_max_r = X_center_0 + Width_0 + (a*threshold_x_0)

            y_min_r = Y_center_0 - (k*threshold_y_0)
            y_max_r = Y_center_0 + (k*threshold_y_0)

            if (x_min_r < X_center_neightbour < x_max_r and y_min_r < Y_center_neightbour < y_max_r ):
                list_of_neightbours_r.append([index_bb])
                try:
                    dict_of_neightbours[str(index_it)+'_r'].append(index_bb)
                except KeyError:
                    dict_of_neightbours[str(index_it)+'_r']= []
                    dict_of_neightbours[str(index_it)+'_r'].append(index_bb)
            else:
                list_of_alones_r.append([index_it, index_bb])
            
    return dict_of_neightbours

def image_mean(x:int, y:int, w:int, h:int, img_path:str) -> float:
    """_summary_

    Args:
        x (int): _description_
        y (int): _description_
        w (int): _description_
        h (int): _description_
        img_path (str): _description_

    Returns:
        float: _description_
    """
    
    image = cv2.imread(img_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    roi = np.mean(gray[y:y + h, x:x + w])

    return roi

def search_voids_bb_neightbours(df_predictions: pd.DataFrame, list_of_dicts: list, h_image:int, w_image:int, img_path) -> list:
    """
    Identifies empty spaces by evaluating each bounding box and it's neightbours. 

    The heuristic of this function draws a "virtual" bounding box beside each bounding box and computes the IoU by iteration over it's neightbours.

    IoU = Intersection over Union (shorturl.at/ituS0)

    If all the calculated IoU (obtained from the neightbours) is less than 10% (0.1), it means that beside the evaluated bounding box, exists an empty space.

    A conditional clause is added with h_image & w_image to avoid surpassing the boundiries of the image in each calculation and avoid irrelevant IoU's.

    Moreover, with image_mean() we double-check the empty spaces. Some empty spaces are not "natural empty spaces", they are just columns or space from each shelf.

    This empty spaces are avoided because do not represent empty *product* spaces.

    Image_mean() returns a floating point value called "roi". Roi is the average pixel value of the a certain space. 

    Args:
        df_predictions (pd.DataFrame): Dataframe with predicted bb obtained with Yolov5.
        list_of_dicts (list): Unified dicts as one list to iterate over. All left and right neightbours added.
        h_image (int): Height of the image.
        w_image (int): Width of the image.
        img_path (_type_): Path to image to be predicted.

    Returns:
        list: List of voids with label and coordinates.
    """
    
    list_of_voids = []

    # Translation of the virtual bounding box. Left = -1 / Right = +1 
    k = 0

    # Void counter
    void_number = 0

    # Iterate over all dicts in list
    for dicts in list_of_dicts:

        # Iterate over key and value pairs of dict (index_a = key // index_b = value pair) - Key = Bounding box being evaluated / Value pair = Neightbours
        for index_a, index_b in dicts.items():

            if index_a[-1] == 'l':
                k = -1
            elif index_a[-1] == 'r':
                k = 1

            # Pick only de integer so the iterator works
            index_a = index_a[0:-2]

            #h_image, w_image = image.shape[0:2] # limits of the image
            w_index_a = df_predictions.loc[int(index_a)][2]
            h_index_a = df_predictions.loc[int(index_a)][3]
            
            # Virtual bounding box to evaluate from neightbours 
            xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 + (k * w_index_a) # Para izquierda: (-1) / Para derecha: 1 / Para arriba: 0
            yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2 
            xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 + (k * w_index_a) # Para izquierda: (-1) / Para derecha: 1 / Para arriba: 0
            yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2 
            boxA = [xA1, yA1, xA2, yA2]

            X_center_A = df_predictions.loc[int(index_a)][0] - k * df_predictions.loc[int(index_a)][2]  # Left X_center - Width  // Right X_center + Width
            Y_center_A = df_predictions.loc[int(index_a)][1]  - k * df_predictions.loc[int(index_a)][3]  # Left Y_center - Width // Right Y_center + Width

            # Limits of the image
            if 0 < xA1 < w_image and 0 < xA2 < w_image and 0 < yA1 < h_image and  0 < yA2 < h_image:
                    trigger = True

                    # Iterate over each neightbour: neightbour vs virtual bounding box(key value)
                    for item in index_b:
                        
                        first_list = []
                        xB1 = df_predictions.loc[item][0] - df_predictions.loc[item][2]/2
                        yB1 = df_predictions.loc[item][1] - df_predictions.loc[item][3]/2
                        xB2 = df_predictions.loc[item][0] + df_predictions.loc[item][2]/2
                        yB2 = df_predictions.loc[item][1] + df_predictions.loc[item][3]/2
                        boxB = [xB1, yB1,xB2, yB2]

                        xA = max(boxA[0], boxB[0])
                        yA = max(boxA[1], boxB[1])
                        xB = min(boxA[2], boxB[2])
                        yB = min(boxA[3], boxB[3])

                        interArea = (xB - xA) * (yB - yA)

                        boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
                        boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

                        iou = interArea / float(boxAArea + boxBArea - interArea)
                        trigger = trigger and (iou < 0.1)

                    if trigger == False:
                        pass
                    else:
                        roi = image_mean(x= int(xA1),y= int(yA1),w= int(w_index_a), h= int(h_index_a), img_path= img_path)
                        if roi < 50:
                            void_number += 1
                            void_text = f'Void_{void_number}'
                            first_list.append(index_a)
                            first_list.append(void_text)
                            first_list.append(xA1)
                            first_list.append(yA1)
                            first_list.append(xA2)
                            first_list.append(yA2)
                            first_list.append(w_index_a)
                            first_list.append(h_index_a)
                            list_of_voids.append(first_list)
                        else:
                            None

    return list_of_voids

def get_df_voids(list_of_dicts_n:list, df_predictions:pd.DataFrame, h_image:int, w_image:int, img_path:str) -> pd.DataFrame:
    """_summary_

    Args:
        list_of_dicts_n (list): _description_
        df_predictions (pd.DataFrame): _description_
        h_image (int): _description_
        w_image (int): _description_
        img_path (str): _description_

    Returns:
        pd.DataFrame: _description_
    """


    # List of dicts(left/right/high) into 1 list
    list_neightbours = list(itertools.chain.from_iterable(list_of_dicts_n))

    # Get void neightbours 
    list_of_voids = search_voids_bb_neightbours(df_predictions= df_predictions, list_of_dicts=list_neightbours, h_image=h_image, w_image=w_image, img_path= img_path)

    # Create dataframe with data(X_center/Y_center/Label) of voids
    df_voids = pd.DataFrame(list_of_voids, columns=['Neightbour','Label','x1','y1','x2','y2', 'Width','Height'])

    return df_voids

def plot_voids_from_df(img_path:str, df_voids: pd.DataFrame) -> None:
    """_summary_

    Args:
        img_path (str): _description_
        df_voids (pd.DataFrame): _description_
    """

    #image_path_bb = os.path.join(folder_path_final, filename)

    # Refactor this with project paths
    img_path = '/home/cremerf/FinalProject/test_neightbours2.jpg'

    # load the image
    image = cv2.imread(img_path)

    # get the coordinates for each index/rectangle
    for i in df_voids.index:
        x1 = int(df_voids.loc[i][2]) 
        y1 = int(df_voids.loc[i][3]) 
        x2 = int(df_voids.loc[i][4])  
        y2 = int(df_voids.loc[i][5])

        # represents the top left corner of rectangle
        start_point=(x1, y1)

        # represents the top right corner of rectangle
        end_point=(x2,y2)

        # # Blue color in BGR
        color = (0, 0, 255)

        # # Line thickness of 5 px
        thickness = 5

        cv2.putText(image,df_voids.loc[i][1],(x1,y1),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)
        # plot the rectangle over the image
        image = cv2.rectangle(image, start_point, end_point, color, thickness)

    # Refactor this with project paths
    cv2.imwrite(filename='test_voids1.jpg', img=image)

In [9]:
img_path = '/home/cremerf/FinalProject/data/images/test/test_7.jpg'

In [10]:
image = cv2.imread(img_path)
h_image, w_image = image.shape[0:2] # limits of the image
df_predictions = yolo.predictions(image=image)

In [15]:
# Get neightbours from 3 ways (right / left / up)
pool = multiprocessing.Pool()
neightbour = 'left'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_left = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

# clean up empty positions
dict_of_neightbours_left = list(filter(None, dict_of_neightbours_left))

pool = multiprocessing.Pool()
neightbour = 'right'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_right = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

dict_of_neightbours_right = list(filter(None, dict_of_neightbours_right))

# Merged 3 separated list of dicts(left/right/high) into 1 list
dicts_neightbours = [dict_of_neightbours_left, dict_of_neightbours_right]
list_neightbours = list(itertools.chain.from_iterable(dicts_neightbours))

# Get void neightbours 
list_of_voids = search_voids_bb_neightbours(df_predictions=df_predictions, list_of_dicts=list_neightbours, h_image=h_image, w_image=w_image, img_path=img_path)

# Create dataframe with data(X_center/Y_center/Label) of voids
df_voids = pd.DataFrame(list_of_voids, columns=['Neightbour','Label','x1','y1','x2','y2', 'Width','Height'])

In [16]:
df_voids

,Neightbour,Label,x1,y1,x2,y2,Width,Height
0,92,Void_1,1960.436034,2224.043330,2068.031620,2368.230877,107.595586,144.187547
1,1145,Void_2,417.097152,2333.991944,544.247887,2409.148132,127.150735,75.156188
2,943,Void_3,1926.983928,2240.429894,2059.756581,2385.724574,132.772653,145.294680


In [11]:
df_predictions

,X_center,Y_center,Width,Height
572,1130.014352,1566.114157,119.923841,347.048751
1059,1073.070552,1188.697504,125.808713,359.978320
581,1603.940973,1564.580640,83.080911,310.984321
585,1751.918793,1564.916821,83.728849,308.152010
1203,759.620471,2645.523212,138.472683,139.737216
...,...,...,...,...
1145,607.823254,2371.570038,127.150735,75.156188
712,2275.615549,2464.040222,214.330290,79.787421
310,2179.846271,1621.568875,53.256595,146.695816
600,856.638524,1615.428113,107.553291,300.801130


In [25]:
for i in df_predictions.iterrows():
    print(i[1]['Width'])

119.92384128570556
125.80871257781982
83.0809112548828
83.72884883880614
138.47268276214598
92.19361267089843
190.13219604492187
127.46199474334716
96.97629261016846
79.78553838729857
85.44594326019286
136.00112514495848
94.18229827880859
136.42744159698486
146.21409187316894
109.55304622650145
146.76536750793457
116.42335968017578
173.46820907592772
91.576238822937
113.11553077697754
197.9141201019287
116.11109790802001
113.4797372817993
113.8411033630371
149.83172149658202
123.10514488220214
136.26537208557127
162.92979011535644
147.9464681625366
173.14146308898924
88.04282970428466
108.25146102905272
109.77232303619384
121.94727344512938
195.494429397583
170.01621894836424
209.02247200012206
140.1248851776123
152.70216407775877
127.87173557281493
173.62910156249998
72.46851968765259
56.45952758789062
134.82749519348144
130.8838394165039
108.90398998260497
144.9230707168579
177.05150070190427
176.2285171508789
77.95790576934814
122.93656768798827
48.99568719863891
144.76308288574216


In [26]:
def generate_colors(ID):
    np.random.seed(10)
    colors = np.random.randint(100,255,size=(nc,3)).tolist()
    return tuple(colors[ID])

In [85]:
df_predictions.head(5)

,X_center,Y_center,Width,Height
572,1130.014352,1566.114157,119.923841,347.048751
1059,1073.070552,1188.697504,125.808713,359.978320
581,1603.940973,1564.580640,83.080911,310.984321
585,1751.918793,1564.916821,83.728849,308.152010
1203,759.620471,2645.523212,138.472683,139.737216


In [84]:
image = cv2.imread(img_path)

for i in df_predictions.iterrows():
    # extract bounding box
    x1 = int(i[1]['X_center']) - int(i[1]['Width'] / 2)
    x2 = int(i[1]['X_center']) + int(i[1]['Width'] / 2)
    y1 = int(i[1]['Y_center']) - int(i[1]['Height'] / 2)
    y2 = int(i[1]['Y_center']) + int(i[1]['Height'] / 2)

    start_point=(x1, y1)

    # represents the top right corner of rectangle
    end_point=(x2,y2)

    # # Blue color in BGR
    color = (0, 255, 0)

    # # Line thickness of 5 px
    thickness = 5

    # plot the rectangle over the image

    image = cv2.rectangle(image, start_point, end_point, color, thickness)

for i in df_voids.index:
    x1 = int(df_voids.loc[i][2]) 
    y1 = int(df_voids.loc[i][3]) 
    x2 = int(df_voids.loc[i][4])  
    y2 = int(df_voids.loc[i][5])

    width = int(df_voids.loc[i][6]/2) 
    height = int(df_voids.loc[i][7]) 

    # represents the top left corner of rectangle
    start_point=(x1, y1)

    # represents the top right corner of rectangle
    end_point=(x2,y2)

    # # Red color in BGR
    color = (0, 0, 255)

    # # Line thickness of 5 px
    thickness = 5


    cv2.rectangle(image, (x1, y1-37),(x2,y2), color, thickness)
    cv2.putText(image, df_voids.loc[i][1], (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
    # plot the rectangle over the image
    cv2.rectangle(image, start_point, end_point, color, thickness)

cv2.imwrite(filename='test_voids2.jpg', img=image)

True

In [ ]:
def plot_voids_from_df(image_name:str, df_voids: pd.DataFrame) -> None:
    """_summary_

    Args:
        img_path (str): _description_
        df_voids (pd.DataFrame): _description_
    """
        # load the image

    img_path = os.path.join(PATHS.UPLOAD_FOLDER, image_name)

    image = cv2.imread(img_path)

    for i in df_predictions.iterrows():
        # extract bounding box
        x1 = int(i[1]['X_center']) - int(i[1]['Width'] / 2)
        x2 = int(i[1]['X_center']) + int(i[1]['Width'] / 2)
        y1 = int(i[1]['Y_center']) - int(i[1]['Height'] / 2)
        y2 = int(i[1]['Y_center']) + int(i[1]['Height'] / 2)

        start_point=(x1, y1)

        # represents the top right corner of rectangle
        end_point=(x2,y2)

        # # Blue color in BGR
        color = (0, 255, 0)

        # # Line thickness of 5 px
        thickness = 5

        # plot the rectangle over the image

        image = cv2.rectangle(image, start_point, end_point, color, thickness)

    for i in df_voids.index:
        x1 = int(df_voids.loc[i][2]) 
        y1 = int(df_voids.loc[i][3]) 
        x2 = int(df_voids.loc[i][4])  
        y2 = int(df_voids.loc[i][5])

        width = int(df_voids.loc[i][6]/2) 
        height = int(df_voids.loc[i][7]) 

        # represents the top left corner of rectangle
        start_point=(x1, y1)

        # represents the top right corner of rectangle
        end_point=(x2,y2)

        # # Red color in BGR
        color = (0, 0, 255)

        # # Line thickness of 5 px
        thickness = 5


        cv2.rectangle(image, (x1, y1-37),(x2,y2), color, thickness)
        cv2.putText(image, df_voids.loc[i][1], (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 3)
        # plot the rectangle over the image
        cv2.rectangle(image, start_point, end_point, color, thickness)

    prediction_path = os.path.join(PATHS.PREDICTIONS, image_name)

    cv2.imwrite(filename= prediction_path, img=image)